In [1]:
import numpy as np
import librosa
import librosa.display
from IPython.display import Audio
import cv2
import matplotlib.pyplot as plt

In [162]:
# Load the spectrogram image using OpenCV
n_fft = 512
hop_length = 64
sr=10000

def reconstruct_audio(image):
    spec_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
    
    # Convert the spectrogram image to a numpy array
    spec_array = np.array(spec_image)
        
    # Denormalise Spectrogram
    spec_array = cv2.normalize(spec_array, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX) #Produces cleaner audio although a lot of frequency content is lost

    # Convert the spectrogram array to a magnitude spectrogram
    spec = librosa.db_to_amplitude(spec_array) 
        
    # Reconstruct the audio signal from the magnitude spectrogram
    y_reconstructed = librosa.feature.inverse.mel_to_audio(spec, sr=sr, n_fft=n_fft, n_iter=1, fmax=sr, hop_length=hop_length)
    
    #return y_reconstructed
    # Save the reconstructed audio signal as a WAV file
    return Audio(y_reconstructed, rate=sr)


In [168]:
reconstruct_audio('color.png')

In [164]:

audio_file = "yes.wav"
y, sr = librosa.load(audio_file, sr=sr, mono=True)
S = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft,
                                           hop_length=hop_length) 
plt.figure(figsize=(10,10))
#plt.axis("off")
S_dB = librosa.power_to_db(S, ref=np.max) 
librosa.display.specshow(S_dB, sr=sr, hop_length=hop_length,
                                 x_axis='time', y_axis='mel', n_fft=n_fft)


C:\Users\kongf\anaconda3\lib\site-packages\librosa\util\decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: 'yes.wav'

In [165]:
from PIL import Image
import glob

# Set the path to the folder containing the images
image_folder = './reconstructed_images/'
# Set the name of the output GIF file
output_folder = "./output/"

# Get a list of all the image file names in the folder
image_files = glob.glob(image_folder + '*.png')
# Sort the file names alphabetically to ensure they are in the correct order
image_files.sort()
def create_final_audio(folder):
    
        audio = np.array([])
        for i, filename in enumerate(image_files):
            spec_image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
#                
            x, y, w, h = 10, 10, 544, 544
#    
            spec_image = spec_image[y:y+h, x:x+w]
#            
#                print(spec_image)
            spec_array = np.array(spec_image)
#                print(spec_array)
#
            spec = librosa.db_to_amplitude(spec_array) 
#        
            y_reconstructed = librosa.feature.inverse.mel_to_audio(spec, sr=sr, n_fft=n_fft, n_iter=1, fmax=sr, hop_length=hop_length)
            c  = np.zeros(2500)
            b  = np.arange(0,1,0.01) 
            a  = np.append(c, b)
            len_a = len(a) 
            flat = np.ones((len(y_reconstructed)-2 * len_a)) 
            rev_a = np.flip(a)
            env = np.append(a,flat)
            env = np.append(env,rev_a)
            final_wav = y_reconstructed * env
            
            #wavefile.write(f"./reconstructed_images/{i}.wav", 10000, y_reconstructed)
            audio = np.append(audio, final_wav)
        return audio
#
#        


In [166]:
final_array = create_final_audio(image_files)

In [167]:
Audio(final_array, rate=sr)

In [74]:
from pydub import AudioSegment

# Set the length of the window in milliseconds
window_length = 100

# Load the first audio file and apply a Hann window to the beginning
audio = AudioSegment.from_file("./reconstructed_images/0.wav")
window = AudioSegment.silent(duration=window_length)
audio = window.overlay(audio[:window_length].fade_out(window_length), position=0)

# Iterate over the remaining audio files, apply a window to the beginning, and concatenate
for i in range(2, num_files+1):
    file_name = f'file{i}.wav'
    audio_i = AudioSegment.from_file(file_name)
    window_i = AudioSegment.silent(duration=window_length)
    audio_i = window_i.overlay(audio_i[:window_length].fade_out(window_length), position=0)
    audio += audio_i

# Save the concatenated audio file
audio.export("output.wav", format="wav")

C:\Users\kongf\anaconda3\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [50]:
import scipy.io.wavfile as wavefile